In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from dateutil.parser import parse
from datetime import datetime
import seaborn as sns

# Table of Contents <a id="toc"></a>
1. [Datasets](#datasets)
2. [Reading, analyzing the datasets](#reading)
   - [2.1. Reading data](#2-1)
   - [2.2. Database Fields Description](#2-2)
   - [2.3. Dataset cleaning](#2-3)
     - [2.3.1. Columns overview](#2-3-1)
	 - [2.3.2. Change column names](#2-3-2)
	 - [2.3.3. Unique values, nan values](#2-3-3)
	 - [2.3.4. Data types](#2-3-4)
	 - [2.3.5. Convert date to datetime](#2-3-5)

# Aviation accidents analysis

In this project we will analyze the reasons why accidents happen in aviation.
An aviation accident is an incident in which an aircraft is damaged or destroyed as a result of a collision, fire, structural failure, or other event. Aviation accidents can be caused by a variety of factors, including mechanical failure, pilot error, adverse weather conditions, and sabotage. Aviation accidents can result in fatalities, injuries, and damage to property.

## I. Datasets <a id="datasets"></a> 
[Return to Table of Contents](#toc)

We'll use 2 datasets from Kaggle:
### Dataset 1. Historical Plane Crash Data
This dataset contains all of the plane crashes that happened throughout history. Data are extracted using web scraping methods in R and preprocessed to create an organized and informative dataset.
All data are collected from Ronan Hubert: [Aviation Accident Bureau Archive]
(https://www.baaa-acro.com/)
This is the dataset [here]:(https://www.kaggle.com/datasets/abeperez/historical-plane-crash-data)

### Dataset 2. Aviation Accident Database & Synopses, up to 2023

The dataset is [here](https://www.kaggle.com/datasets/khsamaha/aviation-accident-database-synopses)
The NTSB aviation accident database [here](https://www.ntsb.gov/Pages/home.aspx) contains information from 1962 and later about civil aviation accidents and selected incidents within the United States, its territories and possessions, and in international waters.

## II. Reading, analyzing the datasets <a id="reading"></a>
[Return to Table of Contents](#toc)

#### Dataset 1 - Historical Plane Crash Data 
This dataset contains all of the plane crashes that happened throughout history. Data are extracted using web scraping methods in R and preprocessed to create an organized and informative dataset.
All data are collected from Ronan Hubert: [Aviation Accident Bureau Archive]
(https://www.baaa-acro.com/)
This is the dataset [here]:(https://www.kaggle.com/datasets/abeperez/historical-plane-crash-data)

### 2.1. Reading data <a id="2-1">
[Return to Table of Contents](#toc)

In [2]:
avioset_aab = pd.read_csv('data/dataset_1_accident_bureau/Plane Crashes.csv')

In [3]:
# 28536 rows × 24 columns
print(avioset_aab.shape)
avioset_aab.head()

(28536, 24)


,Date,Time,Aircraft,Operator,Registration,Flight phase,Flight type,Survivors,Crash site,Schedule,...,Country,Region,Crew on board,Crew fatalities,Pax on board,PAX fatalities,Other fatalities,Total fatalities,Circumstances,Crash cause
0,1918-05-02,NaN,De Havilland DH.4,United States Signal Corps - USSC,AS-32084,Takeoff (climb),Test,No,Airport (less than 10 km from airport),Dayton - Dayton,...,United States of America,North America,2.0,2.0,0.0,0.0,0.0,2,The single engine airplane departed Dayton-McC...,Technical failure
1,1918-06-08,NaN,Handley Page V/1500,Handley Page Aircraft Company Ltd,E4104,Takeoff (climb),Test,Yes,Airport (less than 10 km from airport),Cricklewood - Cricklewood,...,United Kingdom,Europe,6.0,5.0,0.0,0.0,0.0,5,"Assembled at Cricklewood Airfield in May 1918,...",Technical failure
2,1918-06-11,NaN,Avro 504,Royal Air Force - RAF,A8544,Flight,Training,Yes,"Plain, Valley",Abukir - Abukir,...,Egypt,Africa,2.0,1.0,0.0,0.0,0.0,1,The single engine aircraft was completing a lo...,Unknown
3,1918-06-19,NaN,De Havilland DH.4,United States Signal Corps - USSC,AS-32098,Flight,Military,No,Airport (less than 10 km from airport),Wright Patterson AFB-Wright Patterson AFB,...,United States of America,North America,1.0,1.0,0.0,0.0,0.0,1,"Lt. Frank Stuart Patterson, son and nephew of ...",Technical failure
4,1918-06-24,NaN,Breguet 14,French Air Force - Armée de l'Air,AS-4130,Landing (descent or approach),Military,Yes,NaN,NaN,...,France,Europe,NaN,0.0,NaN,0.0,0.0,0,The aircraft crashed iupon landing somewhere i...,Unknown


<div style="background-color: #9df9ef; padding: 10px;"></div>

### 2.2. Database Fields Description <a id="2-2">
[Return to Table of Contents](#toc)

In [4]:
dataColumnsMeaning = pd.read_csv('data/dataset_1_accident_bureau/AAB_DB_Fields_Description.csv', encoding='utf-8')

In [5]:
dataColumnsMeaning.columns

Index(['Field name', 'Description'], dtype='object')

In [6]:
dataColumnsMeaning.shape

(24, 2)

In [7]:
dataColumnsMeaning.style.set_properties(**{'text-align': 'left'})

,Field name,Description
0,date,Date of accident
1,time,Time of accident
2,aircraft,Manufacturer and exact model involved in the accident
3,operator,"The company, organisation or individual operating the aircraft at the time of the accident"
4,registration,The registrationmark applied on the the aircaft at the time of the accident
5,flight_phase,Phase of flight
6,flight_type,"Flight types refer to the various classifications of flights based on their purpose, nature, or the kind of aircraft operation being conducted"
7,survivors,Number of survivors
8,crash_site,Crash site
9,schedule,Schedule


<div style="background-color: #9df9ef; padding: 10px;"></div>

### 2.3. Dataset cleaning <a id="2-3">
[Return to Table of Contents](#toc)

#### 2.3.1. Columns overview <a id="2-3-1">
Let's see what data do we have in all columns. 

In [8]:
avioset_aab.columns

Index(['Date', 'Time', 'Aircraft', 'Operator', 'Registration', 'Flight phase',
       'Flight type', 'Survivors', 'Crash site', 'Schedule', 'MSN', 'YOM',
       'Flight no.', 'Crash location', 'Country', 'Region', 'Crew on board',
       'Crew fatalities', 'Pax on board', 'PAX fatalities', 'Other fatalities',
       'Total fatalities', 'Circumstances', 'Crash cause'],
      dtype='object')

<div style="background-color: #9df9ef; padding: 10px;"></div>

#### 2.3.2. Change column names <a id="2-3-2">
Let's make our column names more Pythonic, which means changing names like Event.Id to event_id.

In [9]:
# We will make a new dataset so as not to lose the original
avioset_aab_low = avioset_aab.copy()

In [10]:
def change_col_names(df, old_symbol, new_symbol):
    df.columns = df.columns.str.lower().str.replace(old_symbol, new_symbol)
    return df

In [11]:
# Changing column names to pythonic
change_col_names(avioset_aab_low, ' ', '_')

# Changing "flight_no." to "flight_no"
avioset_aab_low.rename(columns={'flight_no.': 'flight_no'}, inplace=True)

# Test after the change
avioset_aab_low.columns

Index(['date', 'time', 'aircraft', 'operator', 'registration', 'flight_phase',
       'flight_type', 'survivors', 'crash_site', 'schedule', 'msn', 'yom',
       'flight_no', 'crash_location', 'country', 'region', 'crew_on_board',
       'crew_fatalities', 'pax_on_board', 'pax_fatalities', 'other_fatalities',
       'total_fatalities', 'circumstances', 'crash_cause'],
      dtype='object')

<div style="background-color: #9df9ef; padding: 10px;"></div>

#### 2.3.4. Data types <a id="2-3-4">

In [12]:
# let's check dtypes
avioset_aab_low.dtypes

date                 object
time                 object
aircraft             object
operator             object
registration         object
flight_phase         object
flight_type          object
survivors            object
crash_site           object
schedule             object
msn                  object
yom                 float64
flight_no           float64
crash_location       object
country              object
region               object
crew_on_board       float64
crew_fatalities     float64
pax_on_board        float64
pax_fatalities      float64
other_fatalities    float64
total_fatalities      int64
circumstances        object
crash_cause          object
dtype: object

<div style="background-color: #9df9ef; padding: 10px;"></div>

#### 2.3.5. Convert date to datetime <a id="2-3-5">
- We'll convert date to datetime.
- We'll add 3 new columns 'year', 'month', 'day'.
- We'll delete the 'date' column because we don't need it anymore.

In [13]:
avioset_aab_low.date.unique

<bound method Series.unique of 0        1918-05-02
1        1918-06-08
2        1918-06-11
3        1918-06-19
4        1918-06-24
            ...    
28531    2022-05-20
28532    2022-05-23
28533    2022-05-24
28534    2022-05-29
28535    2022-06-03
Name: date, Length: 28536, dtype: object>

In [14]:
avioset_aab_low['date'] = pd.to_datetime(avioset_aab_low['date'])

In [15]:
avioset_aab_low.date.unique

<bound method Series.unique of 0       1918-05-02
1       1918-06-08
2       1918-06-11
3       1918-06-19
4       1918-06-24
           ...    
28531   2022-05-20
28532   2022-05-23
28533   2022-05-24
28534   2022-05-29
28535   2022-06-03
Name: date, Length: 28536, dtype: datetime64[ns]>

In [16]:
# Let's add column year
avioset_aab_low['year'] = pd.DatetimeIndex(avioset_aab_low['date']).year

In [17]:
# Let's add column month
avioset_aab_low['month'] = pd.DatetimeIndex(avioset_aab_low['date']).month

In [18]:
# Let's add column day
avioset_aab_low['day'] = pd.DatetimeIndex(avioset_aab_low['date']).day

In [19]:
avioset_aab_low['year'].unique()

array([1918, 1919, 1920, 1921, 1922, 1923, 1924, 1925, 1926, 1927, 1928,
       1929, 1930, 1931, 1932, 1933, 1934, 1935, 1936, 1937, 1938, 1939,
       1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950,
       1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961,
       1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972,
       1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983,
       1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994,
       1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005,
       2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,
       2017, 2018, 2019, 2020, 2021, 2022])

In [20]:
avioset_aab_low['month'].unique()

array([ 5,  6,  7,  8,  9, 10, 11, 12,  1,  2,  3,  4])

In [21]:
avioset_aab_low['day'].unique()

array([ 2,  8, 11, 19, 24, 14, 18, 23, 26, 13,  6, 27,  9, 12, 20, 16, 21,
       22, 31, 10,  7, 15,  1,  3,  4, 17, 25, 29,  5, 28, 30])

In [22]:
# let's convert some columns into int
# for col in ["crew_on_board", "crew_fatalities", "pax_on_board", "pax_fatalities", "other_fatalities"]:
#    avioset_aab_low[col] = avioset_aab_low[col].astype(int)

In [23]:
# let's check dtypes again
avioset_aab_low.dtypes

date                datetime64[ns]
time                        object
aircraft                    object
operator                    object
registration                object
flight_phase                object
flight_type                 object
survivors                   object
crash_site                  object
schedule                    object
msn                         object
yom                        float64
flight_no                  float64
crash_location              object
country                     object
region                      object
crew_on_board              float64
crew_fatalities            float64
pax_on_board               float64
pax_fatalities             float64
other_fatalities           float64
total_fatalities             int64
circumstances               object
crash_cause                 object
year                         int32
month                        int32
day                          int32
dtype: object

#### 2.3.3. Unique values, nan values <a id="2-3-3">
Let's look at the unique values and nan values in each column and decide which columns to use for our further analysis.

In [24]:
# View unique values for all columns
def all_columns_unique_values(df):
    for col in df.columns:
        print(f"Unique values in column \'{col}\': \n {df[col].unique()}\n\n **************** \n")

all_columns_unique_values(avioset_aab_low)

Unique values in column 'date': 
 <DatetimeArray>
['1918-05-02 00:00:00', '1918-06-08 00:00:00', '1918-06-11 00:00:00',
 '1918-06-19 00:00:00', '1918-06-24 00:00:00', '1918-07-14 00:00:00',
 '1918-07-18 00:00:00', '1918-08-19 00:00:00', '1918-08-23 00:00:00',
 '1918-08-26 00:00:00',
 ...
 '2022-04-22 00:00:00', '2022-05-06 00:00:00', '2022-05-08 00:00:00',
 '2022-05-11 00:00:00', '2022-05-12 00:00:00', '2022-05-20 00:00:00',
 '2022-05-23 00:00:00', '2022-05-24 00:00:00', '2022-05-29 00:00:00',
 '2022-06-03 00:00:00']
Length: 18562, dtype: datetime64[ns]

 **************** 

Unique values in column 'time': 
 [nan '15H 15M 0S' '12H 0M 0S' ... '3H 21M 0S' '2H 53M 0S' '13H 33M 0S']

 **************** 

Unique values in column 'aircraft': 
 ['De Havilland DH.4' 'Handley Page V/1500' 'Avro 504' ...
 'Cirrus Vision SF50' 'Socata TBM-910' 'Learjet 75']

 **************** 

Unique values in column 'operator': 
 ['United States Signal Corps - USSC' 'Handley Page Aircraft Company Ltd'
 'Royal Air

In [25]:
# View NAN values in all columns
def all_columns_nan_values(df):
    for col in df.columns:
        print(f"All nan values in column \'{col}\': \n {df[col].isna().sum()}\n\n **************** \n")

all_columns_nan_values(avioset_aab_low)

All nan values in column 'date': 
 0

 **************** 

All nan values in column 'time': 
 14587

 **************** 

All nan values in column 'aircraft': 
 1

 **************** 

All nan values in column 'operator': 
 0

 **************** 

All nan values in column 'registration': 
 815

 **************** 

All nan values in column 'flight_phase': 
 638

 **************** 

All nan values in column 'flight_type': 
 57

 **************** 

All nan values in column 'survivors': 
 1297

 **************** 

All nan values in column 'crash_site': 
 383

 **************** 

All nan values in column 'schedule': 
 8946

 **************** 

All nan values in column 'msn': 
 4182

 **************** 

All nan values in column 'yom': 
 5311

 **************** 

All nan values in column 'flight_no': 
 28536

 **************** 

All nan values in column 'crash_location': 
 12

 **************** 

All nan values in column 'country': 
 1

 **************** 

All nan values in column 'region': 
 1



We would like to change data type to int for columns crew_on_board, crew_fatalities, pax_on_board, pax_fatalities, other_fatalities, total_fatalities. 
That's why let's first analyze min and max values.

In [26]:
avioset = avioset.fillna(
    {'crew_on_board':0,
     'crew_fatalities':0,
     'pax_on_board':0,
     'pax_fatalities':0,
     'other_fatalities':0,
     'total_fatalities':0
    })

NameError: name 'avioset' is not defined

# TODO why we decided do choose these columns

#### 2.3.4. Our new dataset <a id="2-3-4">
We will continue the analysis with the most useful data columns
'date', 'aircraft', 'operator', 'flight_phase', 'flight_type', 'survivors', 'crash_site', 'crash_location', 'country', 'region', 'crew_on_board', 'crew_fatalities', 'pax_on_board', 'pax_fatalities', 'other_fatalities','total_fatalities', 'crash_cause'

In [ ]:
# TODO Charts

In [ ]:
avioset = avioset_aab_low[['date', 'aircraft', 'operator', 'flight_phase', 'flight_type', 'survivors', 'crash_site', 'crash_location', 'country', 'region', 'crew_on_board', 'crew_fatalities', 'pax_on_board', 'pax_fatalities', 'other_fatalities','total_fatalities', 'crash_cause']]

print(avioset.shape)
avioset.head()

In [ ]:
avioset.columns

In [ ]:
def max_counts(array_of_columns):   
    for col in array_of_columns:
        print(f"Max value for {col}, {avioset[col].max()}")
        
columns_count_fatalities = ["crew_on_board", "crew_fatalities", "pax_on_board", "pax_fatalities", "other_fatalities", "total_fatalities"]        
max_counts(columns_count_fatalities)

In [ ]:
plt.scatter(avioset['crew_on_board'], avioset['total_fatalities'])
plt.title('Relationship between crew on board and total fatalities')
plt.xlabel('Crew on board')
plt.ylabel('Total fatalities')
plt.show()

#### There is an outlier. Let's see which is this row with outlier

In [ ]:
avioset[avioset.crew_on_board == 1924]

It sounds unreal to have 1924 crew on board.
That's why we'll delete it.

In [ ]:
avioset = avioset[avioset['crew_on_board'] < 1924]

In [ ]:
max_counts(columns_count_fatalities)

In [ ]:
# View NAN values in all columns
all_columns_nan_values(avioset)

In [ ]:
# Nan in survivors we'll fill with "unknown"
avioset = avioset.fillna({'survivors':"unknown"})

In [ ]:
# View NAN values in all columns
all_columns_nan_values(avioset)

#### Flight phases
Picture is from https://docs.flybywiresim.com/pilots-corner/advanced-guides/flight-phases/
![Flight phases](data/images/flight-phases.png)

In [ ]:
avioset.groupby('flight_phase').size().plot()
plt.xticks(rotation = "vertical")
plt.title("Most dangerous flight phases")
plt.show()

In [ ]:
print(avioset.groupby('flight_phase').size().sort_values(ascending=False))

### Conclusion: Most dangerous flight phases are Flight, Landing and Takeoff. They need most concentration.

### Now let's see the 5 operators with the most accidents

In [ ]:
operators_max = avioset.groupby('operator').size().sort_values(ascending=False)
print(operators_max.shape)
operators_max.head()

### Now let's see the 5 operators with the least accidents

In [ ]:
operators_min = avioset.groupby('operator').size().sort_values(ascending=True)
print(operators_min.shape)
operators_min.head()

### Let's investigate the most common cause for accident.
Interestingly, the human factor is the most common cause of accidents

In [ ]:
avioset.groupby('crash_cause').size().plot()
plt.xticks(rotation = "vertical")
plt.title("Accidents by crash cause")
plt.show()

### Let's investigate which countries have most accidents.

In [ ]:
country_max = avioset.groupby('country').size().sort_values(ascending=False)
print(country_max.shape)
country_max.head()

In [ ]:
# Create a vertical bar chart with customizations
ax = country_max[:20].plot.barh(width=0.8, color=['red', 'orange'])

# Customize the plot
ax.set_xlabel('Number of accidents')
ax.set_ylabel('Countries')
ax.set_title('Twenty countries with most accidents')
ax.legend(['Number of accidents', 'country_max[:10]'])

# Show the plot
plt.show()

In [ ]:
# Create a vertical bar chart with customizations
ax = country_max[:10].plot.bar(width=0.8, color=['red', 'orange'])

# Customize the plot
ax.set_xlabel('Number of accidents')
ax.set_ylabel('Countries')
ax.set_title('Ten countries with most accidents')
ax.legend(['Number of accidents', 'country_max[:10]'])

# Show the plot
plt.show()

### Our file became too heavy. That's why we'll save it and we'll continue in part 2.

In [ ]:
avioset.to_csv('data/dataset_1_accident_bureau/aviodata.csv', index=False)

In [ ]:
# test
test = pd.read_csv('data/dataset_1_accident_bureau/aviodata.csv')

In [ ]:
test.head()

<div style="background-color: #9df9ef; padding: 10px;"></div>